In [ ]:
import tensorflow as tf
import zipfile
import os
import matplotlib.pyplot as plt
from snippets import unpack_file, MyCallback, plot_graphs, visualize_convolutions, pick_files_from_directory
import random
import matplotlib.image as mpimg
import numpy as np

In [ ]:
tf.__version__

In [ ]:
DATA_FILE = "intel-image-classification.zip"
DIR = "images"

unpack_file(DATA_FILE, DIR)

TRAIN_DIR = "images/seg_train/seg_train"
TEST_DIR = "images/seg_test/seg_test"

In [ ]:
# Plot sample images

%matplotlib inline
categories = os.listdir(TRAIN_DIR)

IMG_PER_ROW = 4
fig = plt.gcf()
fig.set_size_inches(IMG_PER_ROW*4, len(categories)*4)

for i, category in enumerate(categories):
    
    curr_path = os.path.join(TRAIN_DIR, category)
    path_to_images = pick_files_from_directory(curr_path, k=IMG_PER_ROW)
    
    for j, image_path in enumerate(path_to_images):
        
        sp = plt.subplot(len(categories), IMG_PER_ROW, i*IMG_PER_ROW + j + 1)
        sp.axis("Off")
        
        img = mpimg.imread(image_path)
        plt.imshow(img)
        
plt.show()

In [ ]:
# Process input data
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                  rotation_range=40,
                                                                  width_shift_range=0.2,
                                                                  height_shift_range=0.2,
                                                                  shear_range=0.2,
                                                                  zoom_range=0.2,
                                                                  horizontal_flip=True,
                                                                  fill_mode='nearest')

train_generator = image_generator.flow_from_directory(TRAIN_DIR, target_size=(150, 150), class_mode="categorical", batch_size=32)
test_generator = image_generator.flow_from_directory(TEST_DIR, target_size=(150, 150), class_mode="categorical", batch_size=32)


In [ ]:
# NN from scratch
vanilla_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation="relu", input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(6, activation="softmax")
])

print(vanilla_model.summary())

In [ ]:
tf.keras.backend.clear_session()
            
my_callback = MyCallback(0.7)
vanilla_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001), loss="categorical_crossentropy", metrics=["acc"])
history_vanilla = vanilla_model.fit(train_generator, validation_data=test_generator, epochs=5, verbose=1, callbacks=[my_callback])

In [ ]:
test_metrics = vanilla_model.evaluate_generator(test_generator)

for i in range(len(test_metrics)):
    print("Test {}: {}".format(vanilla_model.metrics_names[i], test_metrics[i]))
    
plot_graphs(history_vanilla, "acc")
plot_graphs(history_vanilla, "loss")

In [ ]:
# NN with transfer learning
pre_trained_model = tf.keras.applications.InceptionV3(input_shape=(150,150,3), 
                                                      include_top=False, 
                                                      weights='imagenet')

for layer in pre_trained_model.layers:
    layer.trainable = False

    
last_layer = pre_trained_model.get_layer("mixed8")
last_output = last_layer.output

x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(1024, activation="relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)                  
x = tf.keras.layers.Dense(6, activation="softmax")(x)           

tl_model = tf.keras.models.Model(pre_trained_model.input, x)

tl_model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), 
              loss = "categorical_crossentropy", 
              metrics = ["acc"])

tl_model.summary()

In [ ]:
tf.keras.backend.clear_session()
            
tl_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001), loss="categorical_crossentropy", metrics=["acc"])
history_tl = tl_model.fit(train_generator, validation_data=test_generator, epochs=5, verbose=1, callbacks=[my_callback])
tl_model(testing_images, testing_labels, verbose=0)


In [ ]:
test_metrics = tl_model.evaluate_generator(test_generator)

for i in range(len(test_metrics)):
    print("Test {}: {}".format(vanilla_model.metrics_names[i], test_metrics[i]))
    
plot_graphs(history_tl, "acc")
plot_graphs(history_tl, "loss")

In [ ]:
# Visualize convolutions
category = "buildings"
model = model_vanilla
img = pick_files_from_directory(os.path.join(TRAIN_DIR, category))[0]

visualize_convolutions(model, img)

In [ ]:
# Predict a file label
id_to_class = {value: key for key, value in train_generator.class_indices.items()}

files = pick_files_from_directory(os.path.join(TEST_DIR, category), k=10)

inputs = []
for file in files: 
    img = tf.keras.preprocessing.image.load_img(file, target_size=(150, 150, 3))
    x = tf.keras.preprocessing.image.img_to_array(img)
    inputs.append(np.expand_dims(x, axis=0))

images = np.vstack(inputs)
labels = vanilla_model.predict_classes(images)

In [ ]:
# Plot it!
fig = plt.gcf()
fig.set_size_inches(15, 50)

i = 0
for file, label in zip(files, labels):
    
    sp = plt.subplot(int(len(files)/2), 2, i+1)
    sp.axis("Off")
    
    img = mpimg.imread(file)
    plt.imshow(img)
    
    plt.title("Predicted: {}".format(id_to_class[label]))
    i+=1
    
plt.show()